In [1]:
import re
import os
import pandas as pd

from google import genai

from utils import *

from dotenv import load_dotenv


pd.set_option('display.max_columns', None)

load_dotenv() 


True

In [2]:
gemini_key = os.getenv("GEMINI_API_KEY")


In [3]:
tests = [
    # email + teléfono + nombre
    "Contactar a Juan Perez al +57 300-123-4567 o jperez@mail.com para validar.",
    # dirección + nombre
    "La propiedad está en 135 Abbott St y el responsable es Maria Lopez.",
    # caso que quede vacío
    "Juan Perez, +57 300-123-4567, jperez@mail.com, 22 Main Street"
]

res = [sanitizar_texto(txt) for txt in tests]

# Salida estructurada

In [4]:
casos_prueba = [
    {
        "id_propiedad": "077-0044-0000",
        "nivel_alerta": "ALTA",
        "motivo": "Inconsistencia detectada. Llamar a Maria Lopez al (301) 555-0199 para confirmar datos.",
        "recomendacion": "Verificar en Calle 123 #45-67 y escribir a mlopez@ejemplo.com para soporte.",
    },
    {
        "id_propiedad": "030-0452-0000",
        "nivel_alerta": "MEDIA",
        "motivo": "Posible error de clasificación. Contactar a Carlos Perez al +57 310-555-0000.",
        "recomendacion": "Solicitar documentación adicional al correo carlos.perez@correo.co y ajustar registro si aplica.",
    },
    {
        "id_propiedad": "015-0467-0000",
        "nivel_alerta": "BAJA",
        "motivo": "Caso atípico por ubicación. Responsable: Ana Gomez, teléfono 3001234567.",
        "recomendacion": "Revisar historial de pagos y enviar notificación a ana.gomez@mail.com con pasos a seguir.",
    },
    {
        "id_propiedad":"077-0144-0000",
        "nivel_alerta": "CRITICA",
        "motivo":"Juan Perez, +57 300-123-4567, jperez@mail.com, Calle 123 #45-67",
        "recomendacion":"Calle 60N #12C-08"
    }
]

for caso in casos_prueba:
    print(salida_estructurada(caso))

{'id_propiedad': '077-0044-0000', 'nivel_alerta': 'ALTA', 'motivo_tecnico': 'Inconsistencia detectada. Llamar a [NAME] al ([PHONE] para confirmar datos', 'recomendacion': 'Verificar en [ADDRESS] y escribir a [EMAIL] para soporte'}
{'id_propiedad': '030-0452-0000', 'nivel_alerta': 'MEDIA', 'motivo_tecnico': 'Posible error de clasificación. Contactar a [NAME] al +[PHONE]', 'recomendacion': 'Solicitar documentación adicional al correo [EMAIL] y ajustar registro si aplica'}
{'id_propiedad': '015-0467-0000', 'nivel_alerta': 'BAJA', 'motivo_tecnico': 'Caso atípico por ubicación. Responsable: [NAME], teléfono [PHONE]', 'recomendacion': 'Revisar historial de pagos y enviar notificación a [EMAIL] con pasos a seguir'}
{'id_propiedad': '077-0144-0000', 'nivel_alerta': 'CRITICA', 'motivo_tecnico': 'SIN_DESCRIPCION_TRAS_SANITIZACION', 'recomendacion': 'SIN_DESCRIPCION_TRAS_SANITIZACION'}


# Explicación con LLM

In [5]:
data_path = "../data/"
file_name = "df_pred_test.csv"
file_full_name = "2024_Property_Tax_Roll.csv"
labels = [ 1,  2,  3,  4,  5,  6,  7, 10, 12, 13, 14, 23, 24, 33, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 82, 83, 84]

df_full = pd.read_csv(data_path + file_full_name)
df_final = pd.read_csv(data_path + file_name)
df_final.shape

(8807, 37)

In [6]:
df_final.head()

,P_ID,TAX_MAP,plat,lot,unit,CLASS,SHORT_DESC,LEVY_CODE_1,SHORT_DESC 1,CIVIC,STREET,SUFFIX,FORMATED_ADDRESS,CITY,ZIP_POSTAL,FIRST_NAME,LAST_NAME,COMPANY,FREE_LINE_2,CIVIC 1,STREET 1,S_SUFFIX,UNIT,CITY 1,STATE,ZIP_POSTAL 1,TOTAL_ASSMT,TOTAL_EXEMPT,TOTAL_TAXES,Property_Location,lon,lat,geo_cluster,p_true,pred_class,y_proba_full,nivel_alerta
0,628.0,005-0315-0000,5.0,315.0,0000,13.0,Residential Vacant Land,NO01,NO01,1.0,Steele,St,1 Steele St,Providence,02906,Nathan,Applegate,NaN,70 Evergreen St,70,Evergreen,St,NaN,Providence,RI,02906,73700.0,0.0,1352.40,POINT (-71.403665419 41.846115898),-71.403665,41.846116,3,0.976598,13,"{13: 0.9765976071357727, 23: 0.022168284282088...",BAJA
1,29585.0,077-0417-0000,77.0,417.0,0000,1.0,Single Family,NO01,NO01,29.0,Windmill,St,29 Windmill St,Providence,02904,Liam,Mallon,NaN,29 Windmill St,29,Windmill,St,NaN,Providence,RI,02904,252300.0,0.0,4629.72,POINT (-71.41789503 41.854803004),-71.417895,41.854803,4,0.574853,1,"{1: 0.5748528242111206, 2: 0.42151743173599243...",CRITICA
2,33245.0,086-0452-0000,86.0,452.0,0000,1.0,Single Family,OO01,OO01,124.0,Freeman Pkwy,NaN,124 Freeman Pkwy,Providence,02906,Helen,Burnham,NaN,Helen Metcalf Burnham Revocable Trust,124,Freeman Pkwy,NaN,NaN,Providence,RI,02906,1483700.0,0.0,15518.76,POINT (-71.393158022 41.837744993),-71.393158,41.837745,3,0.986904,1,"{1: 0.9869039058685303, 2: 0.01208311785012483...",BAJA
3,3804.0,013-0212-0000,13.0,212.0,0000,1.0,Single Family,OO01,OO01,67.0,Manning,St,67 Manning St,Providence,02906,David,Ward,NaN,c/o Tiedemann Advisors,601,"Union St, Suite 3315",NaN,NaN,Seattle,WA,98101,1975100.0,0.0,20658.56,POINT (-71.394910007 41.826489019),-71.394910,41.826489,2,0.831954,1,"{1: 0.8319536447525024, 2: 0.10607580095529556...",ALTA
4,21213.0,059-0264-0000,59.0,264.0,0000,1.0,Single Family,OO01,OO01,74.0,Homer,St,74 Homer St,Providence,02905,STEVEN,DACRUZ,NaN,74 HOMER ST,74,HOMER,ST,NaN,PROVIDENCE,RI,02905-1321,269100.0,71992.0,2061.68,POINT (-71.406669985 41.787739984),-71.406670,41.787740,6,0.966079,1,"{1: 0.9660785794258118, 2: 0.03361643105745315...",MEDIA


#### Generación de paquete para entregar al LLM

In [7]:
# Se realiza el llamado al LLM a 14 registros (permitidos por API key gratis)
df_fool = df_final.sample(frac=0.0016)
dict_alerta_info = df_fool.groupby("nivel_alerta")["p_true"].agg(median="median").to_dict()

df_fool.loc[:,"evidencia_registro"] = df_fool.apply(lambda x: build_evidence_packet(x, df_full, dict_alerta_info), axis = 1)

# Explicación de elección del modelo
- Se decidió elegir el modelo gemini-2.5-flash por encima de otros modelos por:
  * Facilidad de montaje(tiene API gratuita sin algún tipo de crédito).
  * Recibe prompt instruccional, por lo que es fácil y rápido crearlo y modificarlo. Además de que el prompt es muy amigable con el usuario, en el sentido de que se acerca mucho a la manera en como un ser humano podría generar instrucciones para realizar algo.
    + Por ejemplo, un modelo LLaMA requeriría instrucciones más detalladas y con un alto nivel de especificidad para lograr el mismo funcionamiento del modelo elegido que requiere un prompt más sencillo.
  * El modelo es confiable a niveles generales, por lo que el prompt no debe ser muy estructurado ni específico para que fucione bien

- Considerando el tipo de información y respuesta que debe de arrojar el modelo, elegiría gpt-5.2, gemini-3 o claude-4.  Estos modelos no requieren tanto detalle en el prompt, y son ideales realizando tareas complejas y encadenadas.

In [ ]:
evidencia_registro = df_fool["evidencia_registro"].values
client = genai.Client(api_key=gemini_key)
model = "gemini-2.5-flash"
respuesta = []
if False:
    for registro in evidencia_registro:
        print(registro)
        respuesta.append(genera_respuesta(client, registro, model))
else:
    df_fool = pd.read_csv(data_path+"df_respuesta_final.csv")
    responses = df_fool["respuesta_llm"].values
    final_r = []
    for response in responses:
        response = ast.literal_eval(response)
        if len(response) > 0:
            response["motivo"] = sanitizar_texto(response["motivo"])
            response["recomendacion"] = sanitizar_texto(response["recomendacion"])
            final_r.append(response)
    df_fool.loc[:,"respuesta_llm"] = final_r+ [{}]
        